# First Agent

In this notebook, we will create a simple agent that can answer questions about the weather. The agent will use the `langchain` library to interact with a weather API.

In [3]:
import os

In [10]:
from openai import OpenAI

In [11]:
from dotenv import load_dotenv